In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [3]:
"""
Build from class

    >>> from inspect import Signature, Parameter
    >>> params = [  # func(x, y=42, *, z=None)
    ...     Parameter('x', Parameter.POSITIONAL_OR_KEYWORD),
    ...     Parameter('y', Parameter.POSITIONAL_OR_KEYWORD, default=42),
    ...     Parameter('z', Parameter.KEYWORD_ONLY, default=None),
    ... ]
    >>> sig = Signature(params)

Build from callable

    >>> from inspect import signature
    >>> sig = signature(lambda x, y=42, *, z=None: _)

Testing

    >>> print(sig)
    (x, y=42, *, z=None)
    >>> def func(*args, **kwargs):
    ...     bound_values = sig.bind(*args, **kwargs)
    ...     for name, value in bound_values.arguments.items():
    ...         print(name, value)
    >>> func(1, 2, z=3)
    x 1
    y 2
    z 3
    >>> func(1, z=3)
    x 1
    z 3
    >>> func(y=2, x=1)
    x 1
    y 2
    >>> func(1, 2, 3, 4)
    Traceback (most recent call last):
        ...
    TypeError: too many positional arguments
    >>> func(y=2)
    Traceback (most recent call last):
        ...
    TypeError: missing a required argument: 'x'
    >>> func(1, y=2, x=3)
    Traceback (most recent call last):
        ...
    TypeError: multiple values for argument 'x'

"""

doctest.testmod()

TestResults(failed=0, attempted=13)

In [4]:
import inspect


class Structure:
    __signature__ = inspect.Signature()

    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)
        for name, value in bound_values.arguments.items():
            setattr(self, name, value)


class Stock(Structure):
    __signature__ = inspect.signature(lambda name, shares, price: _)


class Point(Structure):
    __signature__ = inspect.signature(lambda x, y: _)


"""

>>> import inspect
>>> print(inspect.signature(Stock))
(name, shares, price)
>>> s1 = Stock('ACME', 100, 490.1)
>>> s2 = Stock('ACME', 100)
Traceback (most recent call last):
    ...
TypeError: missing a required argument: 'price'
>>> s3 = Stock('ACME', 100, 490.1, shares=50)
Traceback (most recent call last):
    ...
TypeError: multiple values for argument 'shares'
"""

doctest.testmod()

TestResults(failed=0, attempted=5)

In [5]:
import inspect


def _make_sig(*names):
    params = [inspect.Parameter(name, inspect.Parameter.POSITIONAL_OR_KEYWORD)
              for name in names]
    return inspect.Signature(params)


class Meta(type):
    def __new__(cls, clsname, bases, clsdict):
        clsdict['__signature__'] = _make_sig(*clsdict.get('_fields',))
        return super().__new__(cls, clsname, bases, clsdict)


class Structure(metaclass=Meta):
    _fields = []

    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)
        for name, value in bound_values.arguments.items():
            setattr(self, name, value)


class Stock(Structure):
    _fields = ('name', 'shares', 'price')


class Point(Structure):
    _fields = ('x', 'y')


"""

>>> import inspect
>>> print(inspect.signature(Stock))
(name, shares, price)
>>> s1 = Stock('ACME', 100, 490.1)
>>> s2 = Stock('ACME', 100)
Traceback (most recent call last):
    ...
TypeError: missing a required argument: 'price'
>>> s3 = Stock('ACME', 100, 490.1, shares=50)
Traceback (most recent call last):
    ...
TypeError: multiple values for argument 'shares'
"""

doctest.testmod()

TestResults(failed=0, attempted=5)